# Code Commenter

This notebook adds comments to code.

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
CODEQWEN_MODEL = "codeqwen"
DEEPSEEK_MODEL = "deepseek-r1"
LLAMA_MODEL = "llama3.2"
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='dummy')

In [ ]:
system_prompt = "You are a coding assistant who adds comments to existing code. You should add doc comments for all classes and functions. "
system_prompt += "You can also add inline comments sparingly where needed to explain complex logic. "
system_prompt += "Do not add inline comments that will be obvious to a knowledgeable developer. "
system_prompt += "Do not modify the code in any way; only add comments. Return only the provided code with your added comments. "
system_prompt += "Do not include summary text or explanations in your response."

In [ ]:
def get_user_prompt(code):
    user_prompt = f"Add comments to the following code.\n\n{code}"
    return user_prompt

In [ ]:
def get_messages(code):
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': get_user_prompt(code)}
    ]
    return messages

In [ ]:
def comment_code(code, model):
    model_name = ""
    
    match model:
        case 'CodeQwen':
            model_name = CODEQWEN_MODEL
        case 'DeepSeek':
            model_name = DEEPSEEK_MODEL
        case 'Llama':
            model_name = LLAMA_MODEL
            
    stream = ollama.chat.completions.create(model=model_name, messages=get_messages(code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("## Code Commenter")
    with gr.Row():
        uncommented = gr.Textbox(label="Code to comment:", value=pi, lines=10)
        commented = gr.Textbox(label="Commented code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["CodeQwen", "DeepSeek", "Llama"], label="Select model", value="CodeQwen")
        comment = gr.Button("Comment code")

    comment.click(comment_code, inputs=[uncommented, model], outputs=[commented])

ui.launch(inbrowser=True)